In [18]:
import pandas as pd
import datetime
import os
import itertools
import json

In [35]:
class GeneradorPlanillaFinanzas:
    def __init__(self):
        pass

    def correr_programa(self):
        diccionario_dfs = self.identificar_tipo_de_archivo_y_cargar_dfs()
        dfs_limpias = self.limpiar_dfs(diccionario_dfs)
        df_completa, df_columnas_utiles = self.unir_dfs(dfs_limpias)

        self.guardar_dfs(df_completa, df_columnas_utiles)
    
    def identificar_tipo_de_archivo_y_cargar_dfs(self):
        identificadores_archivo = {'33': 'SII FACTURAS ELECTRONICAS/EXENTAS', 
                                 '34': 'SII FACTURAS ELECTRONICAS/EXENTAS', 
                                 '61': 'SII NOTAS DE CREDITO',
                                 'reporte': 'ACEPTA',
                                 'presupuesto': 'PRESUPUESTO',
                                 'Documentos recepcion': 'SCI'}

        llaves_identificadoras = list(identificadores_archivo.keys())
                
        diccionario_dfs = {'SII FACTURAS ELECTRONICAS/EXENTAS': None, 
                           'SII NOTAS DE CREDITO': None, 
                           'ACEPTA': None, 
                           'PRESUPUESTO': None, 
                           'SCI': None}

        for nombre_archivo in os.listdir():
            
            if ('.xlsx' in nombre_archivo) or ('.xls' in nombre_archivo) or ('.csv' in nombre_archivo):
                for llave in llaves_identificadoras:
                    if llave in nombre_archivo:
                        identificador_archivo = identificadores_archivo[llave]
                        print(f'Leyendo {nombre_archivo}, es del tipo: {identificador_archivo}')
                        if llave == '33':
                            df = pd.read_csv(nombre_archivo, delimiter = ';')
                        
                        elif llave == '34':
                            df_2 = pd.read_csv(nombre_archivo, delimiter = ';')
                            diccionario_dfs[identificador_archivo] = pd.concat([df, df_2])
                        
                        elif llave == '61':
                            diccionario_dfs[identificador_archivo] = pd.read_csv(nombre_archivo, delimiter = ';')
                        
                        else:
                            diccionario_dfs[identificador_archivo] = pd.read_excel(nombre_archivo)
                        
                        break

        return diccionario_dfs
    
    # Solo se limpian las que no son del SII.    
    def limpiar_dfs(self, diccionario_dfs):
        for nombre_tabla, df in diccionario_dfs.items():
            if nombre_tabla == 'ACEPTA':
                df.rename(columns = {'emisor': 'RUT Emisor', 'folio': 'Folio'}, inplace = True)
            
            elif nombre_tabla == 'PRESUPUESTO':
                df.rename(columns = {'Rut': 'RUT Emisor', 'Folio': 'Folio_abreviado', 'NºDoc.': 'Folio'}, inplace = True)

            elif nombre_tabla == 'SCI':
                df.rename(columns = {'Rut Proveedor': 'RUT Emisor', 'Numero Documento': 'Folio'}, inplace = True)
            
            df['RUT Emisor'] = df['RUT Emisor'].apply(lambda x: x.replace('.', ''))
            
            df.columns = df.columns + f' {nombre_tabla}'

            df[f'RUT Emisor {nombre_tabla}'] = df[f'RUT Emisor {nombre_tabla}'].apply(lambda rut: rut.replace('.', '').upper().strip())

            df['llave_id'] = df[f'RUT Emisor {nombre_tabla}'] + df[f'Folio {nombre_tabla}'].astype(str)
            df.set_index('llave_id', drop = True, inplace = True)

        return diccionario_dfs
    
    def unir_dfs(self, diccionario_dfs_limpias):
        lista_dfs_secuenciales = list(diccionario_dfs_limpias.values())
        df_izquierda = lista_dfs_secuenciales[0]

        for df_derecha in lista_dfs_secuenciales[1:]:
            df_izquierda = pd.merge(df_izquierda, df_derecha, how = 'left', left_index = True, right_index = True)
        
        df_izquierda = df_izquierda[~df_izquierda.index.duplicated(keep = 'first')]

        df_izquierda['tiempo_diferencia'] = pd.to_datetime('today') - pd.to_datetime(df_izquierda['Fecha Docto. SII FACTURAS ELECTRONICAS/EXENTAS'], dayfirst = True)
        df_izquierda['esta_al_dia'] = df_izquierda['tiempo_diferencia'] <= datetime.timedelta(8)

        df_izquierda.to_excel('Prueba.xlsx')

        columnas_a_ocupar = ['Nro. SII FACTURAS ELECTRONICAS', 'RUT Emisor SII FACTURAS ELECTRONICAS', 'razon_social_emisor ACEPTA', 'folio_oc ACEPTA', 'Folio SII FACTURAS ELECTRONICAS', 'Fecha Docto. SII FACTURAS ELECTRONICAS', 'Monto Neto SII FACTURAS ELECTRONICAS', 
                     'Monto Exento SII FACTURAS ELECTRONICAS', 'Monto IVA SII FACTURAS ELECTRONICAS', 'Monto Total SII FACTURAS ELECTRONICAS', 'Fecha Recep. SII FACTURAS ELECTRONICAS', 'Ubic. PRESUPUESTO',
                     'tarea_actual ACEPTA', 'estado_devengo ACEPTA', 'fecha_ingreso_rc ACEPTA', 'estado_nar ACEPTA', 'Motivo SCI', 'tiempo_diferencia', 'esta_al_dia', 'Evento Receptor SII']
        
        df_util = df_izquierda[columnas_a_ocupar]

        return df_izquierda, df_util
    
    def guardar_dfs(self, df_completa, df_columnas_utiles):
        fecha_actual = str(pd.to_datetime('today')).split(' ')[0]
        nombre_archivo = f'PLANILLA DE CONTROL AL {fecha_actual}.xlsx'

        if nombre_archivo in os.listdir():
            with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'a', if_sheet_exists = 'overlay') as writer:
                df_columnas_utiles.to_excel(writer)
        else:
            df_columnas_utiles['Observaciones'] = None
            with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'w') as writer:
                df_columnas_utiles.to_excel(writer)

In [36]:
programa = GeneradorPlanillaFinanzas()

In [37]:
programa.correr_programa()

Leyendo Documentos recepcion (36).xlsx, es del tipo: SCI
Leyendo DTE_RECIBIDOS_61608402_202209_33.csv, es del tipo: SII FACTURAS ELECTRONICAS/EXENTAS
Leyendo DTE_RECIBIDOS_61608402_202209_34.csv, es del tipo: SII FACTURAS ELECTRONICAS/EXENTAS
Leyendo DTE_RECIBIDOS_61608402_202209_61.csv, es del tipo: SII NOTAS DE CREDITO
Leyendo presupuesto_08_09_2022.xlsx, es del tipo: PRESUPUESTO
Leyendo reporte_97017191.xls, es del tipo: ACEPTA
                  Nro. SII FACTURAS ELECTRONICAS/EXENTAS  \
llave_id                                                   
11876578-81422                                       269   
13368432-8686                                        267   
59043540-6126606                                     230   
59051480-2172132                                      71   
59051480-2172379                                     268   
...                                                  ...   
96945670-2384816                                     172   
96945670-2384818         

KeyError: "['Nro. SII FACTURAS ELECTRONICAS', 'RUT Emisor SII FACTURAS ELECTRONICAS', 'Folio SII FACTURAS ELECTRONICAS', 'Fecha Docto. SII FACTURAS ELECTRONICAS', 'Monto Neto SII FACTURAS ELECTRONICAS', 'Monto Exento SII FACTURAS ELECTRONICAS', 'Monto IVA SII FACTURAS ELECTRONICAS', 'Monto Total SII FACTURAS ELECTRONICAS', 'Fecha Recep. SII FACTURAS ELECTRONICAS', 'Evento Receptor SII'] not in index"